In [1]:
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Potrebna popravka, lose napisan pseudokod https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-4-2
# postoji mcode implementiran u Cytoscape alatu
import networkx as nx
from networkx.algorithms.core import core_number

def mcode_vertex_weigting(graph):
    for v in graph.nodes():
        neighbors = list(graph.neighbors(v))
        k_core_graph = graph.subgraph(neighbors)
        k = max(core_number(k_core_graph).values())
        density = nx.density(k_core_graph)
        weight = k * density
        graph.nodes[v]['weight'] = weight
        
def mcode_find_complex(graph, vertex_weight_percentage, seed_vertex, complex_set):
    if seed_vertex in complex_set:
        return

    for v in graph.neighbors(seed_vertex):
        if vertex_weights[v]['weight'] > vertex_weights[seed_vertex]['weight'] * (1 - vertex_weight_percentage):
            complex_set.add(v)
            mcode_find_complex(graph, vertex_weight_percentage, v, complex_set)

def mcode_find_complexes(graph, vertex_weight_percentage):
    complexes = set()
    
    sorted_vertices = sorted(vertex_weights.keys(), key=lambda x: vertex_weights[x], reverse=True)

    for v in sorted_vertices:
        if v not in complexes:
            complex_set = {v}
            mcode_find_compelexes(graph, vertex_weight_percentage, v, complex_set)
            complexes.update(complex_set)

    return complexes
    

In [5]:
if __name__ == "__main__":
    # test example        
    G = nx.Graph()
    G.add_edges_from([(1, 2), (1, 3), (2, 3), (3, 4), (4, 5)])
    mcode_vertex_weigting(G)
    print(G.nodes(data=True))

[(1, {'weight': 1.0}), (2, {'weight': 1.0}), (3, {'weight': 0.3333333333333333}), (4, {'weight': 0}), (5, {'weight': 0})]


In [4]:
import sys
from collections import defaultdict


class MCODE():
    """Class for running and administrating Bader et al.'s MCODE algorithm"""

    def __init__(self, filename, weight_threshold=0.2):
        self.weight_threshold = 1 - weight_threshold
        self.filename = filename
        self.clusters = []

    def cluster(self):
        edges = defaultdict(set)  # node id => neighboring node ids

        # Read edgelist
        with open(self.filename, 'r') as f:
            for line in f:
                a, b = line.split()[:2]
                edges[a].add(b)
                edges[b].add(a)
        print ('## Input graph loaded; %i nodes' % (len(edges),))

        # Clusters list
        clusters = []

        # Stage 1: Vertex Weighting
        print ('## Weighting vertices...')
        weights = dict((v, 1.) for v in edges)
        for i, v in enumerate(edges):
            neighborhood = set((v,)) | edges[v]
            # if node has only one neighbor, we know everything we need to know
            if len(neighborhood) <= 2:
                continue

            # see if larger k-cores exist
            k = 1  # highest valid k-core
            while neighborhood:
                k_core = neighborhood.copy()
                invalid_nodes = True
                while invalid_nodes and neighborhood:
                    invalid_nodes = set(n for n in neighborhood if len(
                        edges[n] & neighborhood) <= k)
                    neighborhood -= invalid_nodes
                k += 1  # on exit, k will be one greater than we want

            # vertex weight = k-core number * density of k-core
            weights[v] = (k - 1) * (sum(len(edges[n] & k_core)
                                        for n in k_core) / (2. * len(k_core)**2))

        # Stage 2: Molecular Complex Prediction
        print('## Molecular complex prediction...')
        unvisited = set(edges)
        num_clusters = 0

        for seed in sorted(weights, key=weights.get, reverse=True):
            if seed not in unvisited:
                continue

            cluster, frontier = set((seed,)), set((seed,))
            w = weights[seed] * self.weight_threshold
            while frontier:
                cluster.update(frontier)
                unvisited -= frontier
                frontier = set(n for n in set.union(
                    *(edges[n] for n in frontier)) & unvisited if weights[n] > w)

            # Haircut: only keep 2-core complexes
            invalid_nodes = True
            while invalid_nodes and cluster:
                invalid_nodes = set(
                    n for n in cluster if len(edges[n] & cluster) < 2)
                cluster -= invalid_nodes

            if cluster:
                # fluff never really seems to improve anything...
                # cluster.update(
                # n for n in set.union(*(edges[c] for c in cluster)) & unvisited
                # if densities[n] > FLUFF_THRESHOLD)

                print (' '.join(cluster))
                num_clusters += 1
                print (num_clusters, len(cluster), seed)
                clusters.append(cluster)

        self.clusters = clusters

    def save_clusters(self, filehandle):
        """Saves clusters, one cluster per line into the input filehandle"""
        with open(filehandle, 'w') as fh:
            for c in self.clusters:
                fh.write(' '.join(c) + "\n")
                
# if __name__ == '__main__':
#     filename = "../data/unweighted_example_network.txt"
#     c = MCODE(filename)
#     c.cluster()
#     c.save_clusters("/tmp/mcode_test.txt")